# PySharpe Scoring Analysis: Value Investing with Technical Validation

This notebook demonstrates how to use PySharpe's scoring analysis to identify potentially undervalued stocks with strong dividend characteristics, while using technical indicators for entry timing.

## Investment Principles

1. **Value Investing**: Focus on fundamentally sound companies trading below their intrinsic value
2. **Income Generation**: Prefer companies with sustainable, growing dividends
3. **Risk Management**: Use technical analysis to improve entry points
4. **Diversification**: Build a balanced portfolio across sectors
5. **Long-term Horizon**: Invest for the long term, avoiding frequent trading

In [ ]:
# Import required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pypfopt import EfficientFrontier, risk_models, expected_returns

# Import PySharpe analysis modules
from pysharpe.analysis.scoring import (
    technical_score,
    dividend_score,
    composite_score,
    validate_data
)
from pysharpe.analysis.visualization import (
    plot_score_distribution,
    plot_score_comparison,
    plot_backtest_results
)
from pysharpe.analysis.backtest import (
    prepare_backtest_data,
    optimize_portfolio,
    simulate_returns
)

## Sample Data

Let's create a sample dataset of dividend-paying stocks with both technical and fundamental metrics.

**Investment Principle**: When analyzing dividend stocks, look for:
- Sustainable payout ratios (< 75%)
- History of dividend increases
- Strong dividend coverage (> 1.5x)
- Reasonable yields (beware of yield traps > 10%)
- Price below technical resistance (SMA200)

In [ ]:
# Create sample data for analysis
data = {
    'Ticker': ['JNJ', 'PG', 'KO', 'PEP', 'MMM', 'CVX', 'XOM', 'VZ', 'T', 'IBM'],
    'Price': [170, 155, 58, 165, 95, 155, 105, 33, 15, 145],
    'YearLow': [155, 140, 52, 150, 85, 140, 95, 30, 13, 130],
    'SMA200': [175, 150, 60, 170, 100, 160, 110, 35, 16, 150],
    'ThreeMonthLow': [165, 145, 55, 160, 90, 145, 100, 31, 14, 135],
    'PER': [18, 25, 22, 24, 15, 12, 10, 8, 6, 15],
    'DividendYield': [0.029, 0.025, 0.031, 0.028, 0.057, 0.040, 0.035, 0.080, 0.095, 0.045],
    'DividendPayout': [0.45, 0.65, 0.70, 0.65, 0.75, 0.50, 0.45, 0.80, 0.90, 0.65],
    'ConsecutiveIncreases': [60, 65, 58, 48, 62, 35, 37, 15, 5, 25],
    'DividendGrowth': [0.06, 0.05, 0.04, 0.07, 0.02, 0.06, 0.05, 0.02, 0.01, 0.03],
    'CoverageRatio': [2.2, 1.8, 1.5, 1.7, 1.3, 2.0, 2.2, 1.2, 1.1, 1.5]
}

df = pd.DataFrame(data)

# Display the first few rows with key metrics
print("Sample Portfolio Analysis")
print("-" * 80)
print("\nKey Metrics:")
display(df[['Ticker', 'Price', 'DividendYield', 'ConsecutiveIncreases', 'CoverageRatio']])

## Calculate Scores

Let's analyze each stock using our scoring system. Note the following principles:

**Technical Analysis**:
- Price near 52-week low suggests potential value
- Price below SMA200 can indicate oversold conditions
- Lower P/E ratios preferred for value

**Dividend Analysis**:
- Long history of increases indicates quality
- Coverage ratio > 1.5 preferred for safety
- Payout ratio < 75% for sustainability
- Moderate yields (3-6%) often more sustainable than very high yields

In [ ]:
# Calculate scores for each stock
tech_scores = []
div_scores = []
composite_scores = []

for _, row in df.iterrows():
    # Calculate technical score
    tech = technical_score(
        row['Price'],
        row['YearLow'],
        row['SMA200'],
        row['ThreeMonthLow'],
        row['PER']
    )
    
    # Calculate dividend score
    div = dividend_score(
        row['DividendYield'],
        row['DividendPayout'],
        row['ConsecutiveIncreases'],
        row['DividendGrowth'],
        row['CoverageRatio']
    )
    
    # Calculate composite score
    comp = composite_score(tech, div, alpha=0.4, beta=0.6)  # Weigh dividend score slightly higher
    
    tech_scores.append(tech)
    div_scores.append(div)
    composite_scores.append(comp)

# Add scores to dataframe
df['TechScore'] = tech_scores
df['DivScore'] = div_scores
df['CompositeScore'] = composite_scores

# Display results sorted by composite score
print("Scoring Results (sorted by composite score)")
print("-" * 80)
display(df.sort_values('CompositeScore', ascending=False)[
    ['Ticker', 'TechScore', 'DivScore', 'CompositeScore']
].style.background_gradient(cmap='RdYlGn'))

## Visualize Score Distribution

Let's analyze the distribution of scores and relationships between technical and dividend metrics.

**Investment Insights**:
- Look for stocks that score well on both technical and dividend metrics
- Be cautious of stocks that score extremely high on just one metric
- Consider sector diversification in final selection
- High technical scores may indicate good entry points
- High dividend scores suggest quality income investments

In [ ]:
# Create visualizations
plt.figure(figsize=(15, 5))

# Plot score distributions
plt.subplot(1, 2, 1)
plot_score_distribution(df)

# Plot technical vs dividend scores
plt.subplot(1, 2, 2)
plot_score_comparison(df)

plt.tight_layout()
plt.show()

# Analysis of potential yield traps
print("\nPotential Yield Traps (High yield but poor scores):")
yield_traps = df[df['DividendYield'] > 0.06].sort_values('CompositeScore')
display(yield_traps[['Ticker', 'DividendYield', 'CoverageRatio', 'CompositeScore']])

## Portfolio Optimization

Now let's use our scores to optimize a portfolio. We'll:
1. Use composite scores to estimate expected returns
2. Generate a correlation matrix for risk management
3. Optimize for the maximum Sharpe ratio

**Investment Principles**:
- Higher scores suggest better risk-adjusted returns
- Diversification reduces portfolio risk
- Position sizing should reflect both risk and potential return
- Regular rebalancing maintains target allocations
- Monitor dividend coverage and payout ratios for changes

In [ ]:
# Prepare data for optimization
mu, cov_matrix = prepare_backtest_data(df, scaling_factor=0.03)  # 3% monthly scaling factor

# Optimize portfolio
weights = optimize_portfolio(mu, cov_matrix)

# Display optimized weights
print("Optimized Portfolio Weights:")
print("-" * 80)
weights_df = pd.DataFrame({
    'Weight': weights,
    'Score': df['CompositeScore'],
    'Dividend Yield': df['DividendYield']
}).round(4)
display(weights_df.sort_values('Weight', ascending=False))

# Calculate portfolio characteristics
port_yield = sum(weights_df['Weight'] * weights_df['Dividend Yield'])
print(f"\nPortfolio Characteristics:")
print(f"Expected Portfolio Yield: {port_yield:.2%}")

# Run backtest simulation
initial_value = 100000  # $100,000 initial investment
portfolio_values = simulate_returns(df, weights, cov_matrix, periods=24, 
                                 initial_value=initial_value)

# Plot backtest results
plot_backtest_results(portfolio_values, 24, initial_value)

## Investment Strategy Summary

Based on our analysis, here are key takeaways for dividend-focused value investing:

1. **Quality First**
   - Focus on companies with long dividend growth histories
   - Ensure adequate dividend coverage and sustainable payout ratios
   - Look for reasonable P/E ratios and strong business models

2. **Value Entry Points**
   - Use technical analysis to identify attractive entry points
   - Consider buying when price is below key moving averages
   - Be patient and wait for quality stocks to reach attractive valuations

3. **Portfolio Management**
   - Diversify across sectors to reduce risk
   - Regular rebalancing to maintain target allocations
   - Monitor fundamental changes in holdings
   - Consider total return (dividend + growth) potential

4. **Risk Management**
   - Avoid yield traps (unusually high yields often signal problems)
   - Watch for deteriorating dividend coverage ratios
   - Monitor payout ratios for sustainability
   - Use position sizing based on risk/reward metrics

5. **Long-term Perspective**
   - Focus on quality over yield
   - Allow time for compound growth
   - Stay invested through market cycles
   - Reinvest dividends when possible

## Extended Analysis: Sector Comparison

Let's analyze a broader set of stocks across different sectors to see how our scoring system performs in various market segments:

1. **Technology**: Growth stocks with emerging dividends
2. **Healthcare**: Defensive stocks with stable dividends
3. **Utilities**: High-yield, regulated businesses
4. **Financial**: Banks and insurance companies
5. **Consumer Staples**: Non-cyclical dividend payers

This diversity helps understand how the scoring system adapts to different business models and dividend policies.

In [ ]:
# Extended dataset with sector diversity
extended_data = {
    'Ticker': [
        # Technology
        'AAPL', 'MSFT', 'CSCO', 'AVGO', 'TSM',
        # Healthcare
        'JNJ', 'PFE', 'ABT', 'MRK', 'UNH',
        # Utilities
        'SO', 'DUK', 'NEE', 'D', 'XEL',
        # Financial
        'JPM', 'BAC', 'BLK', 'MS', 'GS',
        # Consumer Staples
        'PG', 'KO', 'PEP', 'WMT', 'COST'
    ],
    'Sector': [
        'Technology', 'Technology', 'Technology', 'Technology', 'Technology',
        'Healthcare', 'Healthcare', 'Healthcare', 'Healthcare', 'Healthcare',
        'Utilities', 'Utilities', 'Utilities', 'Utilities', 'Utilities',
        'Financial', 'Financial', 'Financial', 'Financial', 'Financial',
        'Consumer', 'Consumer', 'Consumer', 'Consumer', 'Consumer'
    ],
    'Price': [
        175, 330, 52, 850, 90,
        170, 35, 110, 105, 520,
        70, 95, 65, 75, 62,
        145, 28, 650, 85, 320,
        155, 58, 165, 160, 560
    ],
    'YearLow': [
        150, 300, 45, 750, 80,
        155, 30, 100, 95, 480,
        65, 85, 60, 70, 58,
        130, 25, 600, 75, 290,
        140, 52, 150, 145, 520
    ],
    'SMA200': [
        180, 340, 54, 870, 95,
        175, 37, 115, 110, 540,
        72, 98, 67, 77, 64,
        150, 30, 670, 88, 330,
        150, 60, 170, 165, 580
    ],
    'ThreeMonthLow': [
        165, 320, 48, 800, 85,
        165, 32, 105, 100, 500,
        68, 90, 62, 72, 60,
        140, 26, 630, 80, 310,
        145, 55, 160, 155, 550
    ],
    'PER': [
        28, 35, 15, 25, 20,
        18, 12, 22, 15, 30,
        18, 20, 25, 19, 22,
        12, 10, 20, 14, 15,
        25, 22, 24, 28, 40
    ],
    'DividendYield': [
        0.005, 0.008, 0.031, 0.022, 0.018,
        0.029, 0.042, 0.021, 0.028, 0.012,
        0.048, 0.045, 0.038, 0.042, 0.035,
        0.028, 0.032, 0.025, 0.035, 0.022,
        0.025, 0.031, 0.028, 0.015, 0.008
    ],
    'DividendPayout': [
        0.15, 0.25, 0.45, 0.40, 0.35,
        0.45, 0.65, 0.40, 0.45, 0.30,
        0.65, 0.70, 0.60, 0.65, 0.62,
        0.35, 0.40, 0.45, 0.38, 0.35,
        0.65, 0.70, 0.65, 0.45, 0.30
    ],
    'ConsecutiveIncreases': [
        10, 15, 20, 12, 8,
        60, 10, 45, 12, 15,
        20, 25, 28, 15, 18,
        10, 8, 12, 7, 9,
        65, 58, 48, 45, 18
    ],
    'DividendGrowth': [
        0.08, 0.10, 0.05, 0.15, 0.12,
        0.06, 0.03, 0.08, 0.07, 0.09,
        0.03, 0.04, 0.06, 0.04, 0.05,
        0.05, 0.04, 0.08, 0.06, 0.07,
        0.05, 0.04, 0.07, 0.06, 0.08
    ],
    'CoverageRatio': [
        6.5, 4.0, 2.2, 2.5, 2.8,
        2.2, 1.5, 2.5, 2.2, 3.3,
        1.5, 1.4, 1.6, 1.5, 1.6,
        2.8, 2.5, 2.2, 2.6, 2.8,
        1.8, 1.5, 1.7, 2.2, 3.3
    ]
}

# Create extended DataFrame
df_extended = pd.DataFrame(extended_data)

# Calculate scores for extended dataset
tech_scores = []
div_scores = []
composite_scores = []

for _, row in df_extended.iterrows():
    tech = technical_score(row['Price'], row['YearLow'], row['SMA200'],
                         row['ThreeMonthLow'], row['PER'])
    div = dividend_score(row['DividendYield'], row['DividendPayout'],
                       row['ConsecutiveIncreases'], row['DividendGrowth'],
                       row['CoverageRatio'])
    comp = composite_score(tech, div, alpha=0.4, beta=0.6)
    
    tech_scores.append(tech)
    div_scores.append(div)
    composite_scores.append(comp)

df_extended['TechScore'] = tech_scores
df_extended['DivScore'] = div_scores
df_extended['CompositeScore'] = composite_scores

# Display sector averages
print("Sector Analysis")
print("-" * 80)
sector_analysis = df_extended.groupby('Sector').agg({
    'DividendYield': 'mean',
    'ConsecutiveIncreases': 'mean',
    'TechScore': 'mean',
    'DivScore': 'mean',
    'CompositeScore': 'mean'
}).round(3)

display(sector_analysis.style.background_gradient(cmap='RdYlGn'))

In [ ]:
# Create sector-based visualizations
plt.figure(figsize=(15, 10))

# Scatter plot by sector
plt.subplot(2, 1, 1)
sectors = df_extended['Sector'].unique()
colors = plt.cm.Set3(np.linspace(0, 1, len(sectors)))

for sector, color in zip(sectors, colors):
    mask = df_extended['Sector'] == sector
    plt.scatter(df_extended[mask]['TechScore'], 
               df_extended[mask]['DivScore'],
               label=sector, color=color, alpha=0.7)

plt.xlabel('Technical Score')
plt.ylabel('Dividend Score')
plt.title('Score Distribution by Sector')
plt.legend()
plt.grid(True, alpha=0.3)

# Box plot of composite scores by sector
plt.subplot(2, 1, 2)
sns.boxplot(data=df_extended, x='Sector', y='CompositeScore')
plt.xticks(rotation=45)
plt.title('Composite Score Distribution by Sector')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Display top performers in each sector
print("\nTop Performers by Sector:")
print("-" * 80)
for sector in sectors:
    sector_df = df_extended[df_extended['Sector'] == sector]
    top = sector_df.nlargest(2, 'CompositeScore')
    print(f"\n{sector}:")
    display(top[['Ticker', 'DividendYield', 'ConsecutiveIncreases', 
                 'TechScore', 'DivScore', 'CompositeScore']])

## Sector-Specific Investment Insights

Our extended analysis reveals important sector-specific characteristics:

1. **Technology Sector**
   - Lower current yields but higher dividend growth rates
   - Strong coverage ratios due to high cash flows
   - Shorter dividend history but rapid growth
   - Higher P/E ratios typical for growth stocks

2. **Healthcare Sector**
   - Defensive characteristics with stable dividends
   - Strong balance sheets and consistent payouts
   - Moderate yields with good growth potential
   - Less sensitive to economic cycles

3. **Utilities Sector**
   - Highest average yields
   - Most stable payout ratios
   - Lower growth rates but very consistent
   - More sensitive to interest rates

4. **Financial Sector**
   - Yields vary with interest rate cycles
   - Payout ratios controlled by regulations
   - Strong correlation with economic cycles
   - Good dividend growth in strong economies

5. **Consumer Staples**
   - Very long dividend histories
   - Stable but moderate yields
   - Strong brand value supporting margins
   - Defensive characteristics in downturns

These sector characteristics should inform portfolio construction and risk management strategies.

In [ ]:
# Import Altair for interactive visualizations
import altair as alt
# Enable max rows setting for Altair
alt.data_transformers.disable_max_rows()

# Create base scatter plot with interactive selection
selection = alt.selection_point(fields=['Sector'], bind='legend')

# Create interactive score comparison chart
base = alt.Chart(df_extended).encode(
    x=alt.X('TechScore:Q', scale=alt.Scale(domain=[0, 1])),
    y=alt.Y('DivScore:Q', scale=alt.Scale(domain=[0, 1])),
    color='Sector:N',
    tooltip=['Ticker', 'Sector', 'DividendYield', 'ConsecutiveIncreases', 
             'TechScore', 'DivScore', 'CompositeScore']
)

scatter = base.mark_circle(size=100).encode(
    opacity=alt.condition(selection, alt.value(0.9), alt.value(0.2))
).add_params(selection)

# Add regression lines for each sector
regression = base.transform_regression(
    'TechScore', 'DivScore', groupby=['Sector']
).mark_line(size=2).encode(
    opacity=alt.condition(selection, alt.value(0.9), alt.value(0.2))
)

# Combine scatter and regression
score_comparison = (scatter + regression).properties(
    width=600,
    height=400,
    title='Technical vs Dividend Scores by Sector (Interactive)'
)

# Display the interactive visualization
score_comparison

In [ ]:
# Create a dividend yield analysis visualization
yield_chart = alt.Chart(df_extended).mark_circle(size=100).encode(
    x=alt.X('DividendYield:Q', 
            scale=alt.Scale(domain=[0, df_extended['DividendYield'].max() * 1.1]),
            axis=alt.Axis(format='%')),
    y=alt.Y('CoverageRatio:Q'),
    size='ConsecutiveIncreases:Q',
    color='Sector:N',
    tooltip=['Ticker', 'DividendYield', 'CoverageRatio', 'ConsecutiveIncreases']
).properties(
    width=600,
    height=400,
    title='Dividend Yield vs Coverage Ratio (size = years of increases)'
)

# Add a rule to show "danger zone" for coverage ratio
danger_zone = alt.Chart(pd.DataFrame({'y': [1.5]})).mark_rule(
    strokeDash=[5, 5],
    color='red',
    opacity=0.5
).encode(y='y:Q')

# Combine charts
yield_analysis = yield_chart + danger_zone

# Display the visualization
yield_analysis

In [ ]:
# Create a heatmap of composite scores vs market metrics
heatmap_data = df_extended.copy()
heatmap_data['PERBin'] = pd.qcut(df_extended['PER'], q=5, labels=['Very Low', 'Low', 'Medium', 'High', 'Very High'])
heatmap_data['YieldBin'] = pd.qcut(df_extended['DividendYield'], q=5, labels=['Very Low', 'Low', 'Medium', 'High', 'Very High'])

# Calculate average composite score for each combination
heatmap_df = heatmap_data.groupby(['PERBin', 'YieldBin'])['CompositeScore'].mean().reset_index()

# Create heatmap
heatmap = alt.Chart(heatmap_df).mark_rect().encode(
    x='PERBin:O',
    y='YieldBin:O',
    color=alt.Color('CompositeScore:Q', scale=alt.Scale(scheme='viridis')),
    tooltip=['PERBin', 'YieldBin', alt.Tooltip('CompositeScore:Q', format='.3f')]
).properties(
    width=400,
    height=300,
    title='Composite Score Heatmap: P/E Ratio vs Dividend Yield'
)

# Add text values
text = alt.Chart(heatmap_df).mark_text(color='white').encode(
    x='PERBin:O',
    y='YieldBin:O',
    text=alt.Text('CompositeScore:Q', format='.2f')
)

# Combine heatmap and text
composite_heatmap = heatmap + text

# Display the visualization
composite_heatmap

In [ ]:
# Create a stream graph of dividend metrics
stream_data = df_extended.melt(
    id_vars=['Ticker', 'Sector'],
    value_vars=['DividendYield', 'DividendPayout', 'DividendGrowth'],
    var_name='Metric',
    value_name='Value'
)

# Normalize values for better comparison
for metric in ['DividendYield', 'DividendPayout', 'DividendGrowth']:
    max_val = stream_data[stream_data['Metric'] == metric]['Value'].max()
    stream_data.loc[stream_data['Metric'] == metric, 'Value'] /= max_val

stream = alt.Chart(stream_data).mark_area().encode(
    x=alt.X('Ticker:N', sort='-y'),
    y=alt.Y('Value:Q', stack='center', axis=alt.Axis(title='Normalized Value')),
    color=alt.Color('Metric:N', scale=alt.Scale(scheme='category10')),
    tooltip=['Ticker', 'Metric', alt.Tooltip('Value:Q', format='.2f')]
).properties(
    width=800,
    height=400,
    title='Normalized Dividend Metrics Comparison'
).interactive()

# Display the visualization
stream

## Interactive Visualization Analysis

The Altair visualizations above provide several key insights:

1. **Score Comparison Plot**
   - Interactive legend for filtering sectors
   - Regression lines show sector-specific relationships
   - Tooltip details for deep-dive analysis
   - Clear clustering of sectors in score space

2. **Dividend Safety Analysis**
   - Bubble size represents dividend history length
   - Red line shows minimum safe coverage ratio
   - Helps identify potential yield traps
   - Shows sector-specific yield patterns

3. **Composite Score Heatmap**
   - Shows relationship between P/E and yield
   - Darker colors indicate better composite scores
   - Helps identify sweet spots for value
   - Reveals potential market inefficiencies

4. **Dividend Metrics Stream**
   - Normalized comparison across metrics
   - Shows relative strengths by company
   - Identifies balanced dividend profiles
   - Reveals sector-specific patterns

These interactive visualizations help in:
- Identifying potential investments
- Screening for yield traps
- Understanding sector characteristics
- Balancing portfolio allocations

# Evidence-Based Investment Principles

## Core Principles Based on Academic Research

1. **Total Return Focus**
   - Don't chase yield alone; focus on total return (dividends + capital appreciation)
   - Higher yields often signal higher risk
   - Historical evidence shows dividend-paying stocks outperform primarily due to quality factors, not yield itself

2. **Risk Management Fundamentals**
   - Diversification across sectors and factors is crucial
   - Position sizing should consider correlation benefits
   - No single stock should dominate portfolio risk
   - Systematic risk factors explain most returns

3. **Quality Over Yield**
   - Key quality indicators:
     - Strong balance sheets (low debt-to-equity)
     - High return on invested capital (ROIC)
     - Sustainable payout ratios (< 75% for most industries)
     - Growing free cash flow coverage
     - Competitive advantages (moats)

4. **Dividend Growth > High Yield**
   - Research shows dividend growth stocks outperform high-yield stocks long-term
   - Look for:
     - 5-10 years of consistent increases
     - Earnings growth supporting dividend growth
     - Payout ratio leaving room for growth
     - Strong return on equity (ROE)

5. **Valuation Discipline**
   - Price matters: even great companies can be poor investments if overvalued
   - Use multiple valuation metrics:
     - P/E relative to growth rate and sector
     - EV/EBITDA for operating performance
     - Free cash flow yield
     - Dividend discount models

6. **Portfolio Construction Best Practices**
   - Core principles:
     - Broad diversification (15-30 stocks minimum)
     - Sector constraints (max 25% per sector)
     - Factor exposure management
     - Regular rebalancing (semi-annual or annual)
     - Tax efficiency consideration

7. **Common Pitfalls to Avoid**
   - Yield traps (unusually high yields often signal distress)
   - Over-concentration in defensive sectors
   - Ignoring interest rate sensitivity
   - Neglecting international diversification
   - Emotional decision-making during market stress

## Implementation Guidelines

1. **Initial Screening**
   - Start with quality metrics
   - Then check dividend sustainability
   - Finally apply valuation criteria
   - Use technical analysis only for entry timing

2. **Monitoring Process**
   - Track fundamental changes quarterly
   - Review dividend coverage ratios
   - Monitor payout ratio trends
   - Check competitor positions
   - Evaluate industry dynamics

3. **Risk Management Protocol**
   - Set position size limits (e.g., 5% maximum)
   - Maintain sector diversification
   - Monitor factor exposures
   - Set clear sell criteria:
     - Dividend cut or freeze
     - Deteriorating fundamentals
     - Excessive valuation
     - Better opportunities elsewhere

In [ ]:
# Update our scoring weights based on evidence-based principles
DEFAULT_TECH_WEIGHTS = {
    'P_YL': 0.15,        # Reduced weight for price/52-week low (less predictive)
    'P_SMA200': 0.15,    # Reduced weight for technical factors
    'P_ML3': 0.10,       # Reduced weight for short-term price movements
    'PER': 0.60         # Increased weight for fundamental valuation
}

DEFAULT_DIV_WEIGHTS = {
    'yield': 0.15,       # Reduced weight for pure yield
    'payout': 0.25,      # Increased weight for sustainability
    'consec': 0.25,      # Increased weight for consistency
    'growth': 0.25,      # Increased weight for growth
    'coverage': 0.10
}

# Recalculate scores with updated weights
tech_scores = []
div_scores = []
composite_scores = []

for _, row in df_extended.iterrows():
    tech = technical_score(
        row['Price'],
        row['YearLow'],
        row['SMA200'],
        row['ThreeMonthLow'],
        row['PER'],
        weights=DEFAULT_TECH_WEIGHTS
    )
    
    div = dividend_score(
        row['DividendYield'],
        row['DividendPayout'],
        row['ConsecutiveIncreases'],
        row['DividendGrowth'],
        row['CoverageRatio'],
        weights=DEFAULT_DIV_WEIGHTS
    )
    
    # Adjust composite score to favor fundamental factors
    comp = composite_score(tech, div, alpha=0.3, beta=0.7)
    
    tech_scores.append(tech)
    div_scores.append(div)
    composite_scores.append(comp)

# Update scores in dataframe
df_extended['TechScore_Updated'] = tech_scores
df_extended['DivScore_Updated'] = div_scores
df_extended['CompositeScore_Updated'] = composite_scores

# Compare original vs updated scores
comparison = df_extended[['Ticker', 'Sector', 
                         'CompositeScore', 'CompositeScore_Updated']].copy()
comparison['Score_Change'] = comparison['CompositeScore_Updated'] - comparison['CompositeScore']

print("Impact of Evidence-Based Weight Adjustments")
print("-" * 80)
display(comparison.sort_values('Score_Change', ascending=False)
        .style.background_gradient(subset=['Score_Change'], cmap='RdYlGn'))